In [102]:
#loading necessary libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score, cross_val_predict
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc
import pandas as pd
from sklearn import ensemble
import random
import xgboost as xgb
from scipy.stats import rankdata



In [35]:
#importing the datasets
train = pd.read_csv("train_1.csv")
test = pd.read_csv("test_1.csv")
submission = pd.read_csv("Sample_Solution_Yrq0zGh.csv")
print "Train dataset dimensions:", train.shape
print "Test dataset dimensions:", test.shape

Train dataset dimensions: (69713, 25)
Test dataset dimensions: (30037, 25)


In [36]:
#making copies
train_1 = train.copy()
test_1 = test.copy()

In [37]:
train.head()

,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,...,EMI,Var1,Approved,is_male,age,DOB_month,DOB_year,Lead_Creation_day,Lead_Creation_month,Lead_Creation_year
0,C10001,A,Others,A,4.0,2000.0,B001,P,N,S122,...,NaN,0,0,0,37.0,7.0,1979.0,15,7,2016
1,C10003,A,COM0000002,C,1.0,3500.0,B002,P,Y,S122,...,953.0,10,0,1,30.0,12.0,1986.0,4,7,2016
2,Others,C,Others,C,4.0,2250.0,B003,G,Y,S143,...,NaN,0,0,1,34.0,12.0,1982.0,19,7,2016
3,Others,C,Others,A,4.0,3500.0,B003,G,Y,S143,...,NaN,7,0,1,27.0,1.0,1989.0,9,7,2016
4,C10002,A,Others,A,4.0,10000.0,B001,P,Y,S134,...,NaN,10,0,1,31.0,4.0,1985.0,20,7,2016


In [38]:
train_1.dtypes,test_1.dtypes

(City_Code                               object
 City_Category                           object
 Employer_Code                           object
 Employer_Category1                      object
 Employer_Category2                     float64
 Monthly_Income                         float64
 Customer_Existing_Primary_Bank_Code     object
 Primary_Bank_Type                       object
 Contacted                               object
 Source                                  object
 Source_Category                         object
 Existing_EMI                           float64
 Loan_Amount                            float64
 Loan_Period                            float64
 Interest_Rate                          float64
 EMI                                    float64
 Var1                                     int64
 Approved                                 int64
 is_male                                  int64
 age                                    float64
 DOB_month                              

## Train Data Manipulation

In [39]:
label = train_1['Approved']

dropCols = ['Approved']
train_1.drop(dropCols, axis=1, inplace = True)

In [40]:
le = LabelEncoder()
train_1['City_encoded'] = le.fit_transform(train_1['City_Code'])

In [41]:
y_train = label
X_train = pd.get_dummies(train_1)

In [50]:
X_train.dtypes

Employer_Category2     float64
Monthly_Income         float64
Existing_EMI           float64
Loan_Amount            float64
Loan_Period            float64
Interest_Rate          float64
EMI                    float64
Var1                     int64
is_male                  int64
age                    float64
DOB_month              float64
DOB_year               float64
Lead_Creation_day        int64
Lead_Creation_month      int64
Lead_Creation_year       int64
City_encoded             int64
City_Code_C10001         uint8
City_Code_C10002         uint8
City_Code_C10003         uint8
City_Code_C10004         uint8
City_Code_C10005         uint8
City_Code_C10006         uint8
City_Code_C10007         uint8
City_Code_C10008         uint8
City_Code_C10009         uint8
City_Code_C10010         uint8
City_Code_C10011         uint8
City_Code_C10012         uint8
City_Code_C10013         uint8
City_Code_C10014         uint8
                        ...   
Primary_Bank_Type_G      uint8
Primary_

## Test Data Manipulation

In [43]:
test_1.dtypes

ID                                      object
City_Code                               object
City_Category                           object
Employer_Code                           object
Employer_Category1                      object
Employer_Category2                     float64
Monthly_Income                         float64
Customer_Existing_Primary_Bank_Code     object
Primary_Bank_Type                       object
Contacted                               object
Source                                  object
Source_Category                         object
Existing_EMI                           float64
Loan_Amount                            float64
Loan_Period                            float64
Interest_Rate                          float64
EMI                                    float64
Var1                                     int64
is_male                                  int64
age                                    float64
DOB_month                              float64
DOB_year     

In [44]:
test_1['ID'].to_csv("test_ids.csv", index = False)

In [45]:
dropCols = ['ID']
test_1.drop(dropCols, axis=1, inplace = True)

In [46]:
test_1['City_encoded'] = le.fit_transform(test_1['City_Code'])

In [47]:
X_test = pd.get_dummies(test_1)
missingCols = list(set(X_train.columns)-set(X_test.columns))
for col in missingCols:
    X_test[col] = 0
X_test = X_test[X_train.columns]
assert X_train.columns.equals(X_test.columns)

In [49]:
X_test.dtypes

Employer_Category2     float64
Monthly_Income         float64
Existing_EMI           float64
Loan_Amount            float64
Loan_Period            float64
Interest_Rate          float64
EMI                    float64
Var1                     int64
is_male                  int64
age                    float64
DOB_month              float64
DOB_year               float64
Lead_Creation_day        int64
Lead_Creation_month      int64
Lead_Creation_year       int64
City_encoded             int64
City_Code_C10001         uint8
City_Code_C10002         uint8
City_Code_C10003         uint8
City_Code_C10004         uint8
City_Code_C10005         uint8
City_Code_C10006         uint8
City_Code_C10007         uint8
City_Code_C10008         uint8
City_Code_C10009         uint8
City_Code_C10010         uint8
City_Code_C10011         uint8
City_Code_C10012         uint8
City_Code_C10013         uint8
City_Code_C10014         uint8
                        ...   
Primary_Bank_Type_G      uint8
Primary_

# Model Building

In [56]:
test_ids = pd.read_csv("test_ids.csv", header = None)
test_ids = list(test_ids.iloc[:,0])

## XGBoost

#### XGB1

In [55]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'logloss',
          'eta':0.02, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 400

params['seed'] = 123 
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
# xgb.cv(params, dtrain, num_rounds, nfold=4)
# exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds = clf.predict(dtest)

In [60]:
submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_1.csv", index = False)

#### XGB1.1

In [61]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.02, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 400

params['seed'] = 1234
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
# xgb.cv(params, dtrain, num_rounds, nfold=4)
# exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_1_1.csv", index = False)


#### XGB1.2

In [62]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.02, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 400

params['seed'] = 12345
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
# xgb.cv(params, dtrain, num_rounds, nfold=4)
# exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_1_2.csv", index = False)

#### XGB1.3

In [63]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.02, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 400

params['seed'] = 123456
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
# xgb.cv(params, dtrain, num_rounds, nfold=4)
# exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_1_3.csv", index = False)


#### XGB1.4

In [64]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.02, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 400

params['seed'] = 1234567
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
# xgb.cv(params, dtrain, num_rounds, nfold=4)
# exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_1_4.csv", index = False)

#### XGB2

In [65]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.01, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 800

params['seed'] = 123
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
#xgb.cv(params, dtrain, num_rounds, nfold=4)
#exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_2.csv", index = False)

#### XGB2.1

In [66]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.01, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 800

params['seed'] = 1234 
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
#xgb.cv(params, dtrain, num_rounds, nfold=4)
#exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_2_1.csv", index = False)

#### XGB2.2

In [67]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.01, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 800

params['seed'] = 12345 
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
#xgb.cv(params, dtrain, num_rounds, nfold=4)
#exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_2_2.csv", index = False)


#### XGB2.3

In [68]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.01, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 800

params['seed'] = 123456 
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
#xgb.cv(params, dtrain, num_rounds, nfold=4)
#exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_2_3.csv", index = False)

#### XGB2.4

In [70]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.01, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 800

params['seed'] = 1234567
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
#xgb.cv(params, dtrain, num_rounds, nfold=4)
#exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_2_4.csv", index = False)

#### RandomForest

In [97]:
test_1.dtypes

City_Code                               object
City_Category                           object
Employer_Code                           object
Employer_Category1                      object
Employer_Category2                     float64
Monthly_Income                         float64
Customer_Existing_Primary_Bank_Code     object
Primary_Bank_Type                       object
Contacted                               object
Source                                  object
Source_Category                         object
Existing_EMI                           float64
Loan_Amount                            float64
Loan_Period                            float64
Interest_Rate                          float64
EMI                                    float64
Var1                                     int64
is_male                                  int64
age                                    float64
DOB_month                              float64
DOB_year                               float64
Lead_Creation

In [98]:
y_train = label
X_train = pd.get_dummies(train_1)


X_test = pd.get_dummies(test_1)
missingCols = list(set(X_train.columns)-set(X_test.columns))
for col in missingCols:
    X_test[col] = 0
X_test = X_test[X_train.columns]
assert X_train.columns.equals(X_test.columns)

# # Modeling
X_train_2 = X_train.fillna(-999)
X_test_2 = X_test.fillna(-999)



In [99]:
# from sklearn.cross_validation import KFold
# kf = KFold(len(X_train_2), n_folds=4)
# scores = cross_val_score(clf, X_train_2, y_train, scoring='roc_auc', cv=kf)
seeds = [123,1234,5245326,6536,75]
numbers = [1,2,3,4,5]

for i in range(5): 
    clf = RandomForestClassifier(n_estimators=360, max_depth=9, criterion = 'entropy', min_samples_split=2, bootstrap = False, n_jobs=-1, random_state=seeds[i])
    clf.fit(X_train_2, y_train)
    test_preds = clf.predict_proba(X_test_2)[:,1]
    print("RF %s done" % i)

    submission = pd.DataFrame({'ID':test['ID'], 'Approved':test_preds})
    submission = submission[['ID', 'Approved']]
    submission.to_csv("submissions/rf_%s.csv" % str(numbers[i]), index = False)

RF 0 done
RF 1 done
RF 2 done
RF 3 done
RF 4 done


## Post Model Building

In [85]:
test_ids = pd.read_csv("test_ids.csv", names = ['ID'])

In [86]:
test_ids.head()

,ID
0,APPA70109647212
1,APPB10687939341
2,APPC80449411414
3,APPD30665094501
4,APPE80379821637


#### Combining predictions - rf

In [100]:
preds = pd.read_csv("submissions/rf_1.csv")
preds['ID'] = test_ids['ID']

preds2 = pd.read_csv("submissions/rf_2.csv")
preds3 = pd.read_csv("submissions/rf_3.csv")
preds4 = pd.read_csv("submissions/rf_4.csv")
preds5 = pd.read_csv("submissions/rf_5.csv")

preds['Approved'] = rankdata(preds['Approved'], method='ordinal')
preds2['Approved'] = rankdata(preds2['Approved'], method='ordinal')
preds3['Approved'] = rankdata(preds3['Approved'], method='ordinal')
preds4['Approved'] = rankdata(preds4['Approved'], method='ordinal')
preds4['Approved'] = rankdata(preds5['Approved'], method='ordinal')

preds['Approved'] = 0.3 * (preds['Approved'] + 
                            preds2['Approved'] + 
                            preds3['Approved'] + 
                            preds4['Approved'] + 
                            preds5['Approved'])

preds.to_csv("submissions/ensembles/rf_ens.csv", index = False)

#### Combining predictions - xgboost1

In [94]:
preds = pd.read_csv("submissions/xgb_1.csv")
preds['ID'] = test_ids['ID']

preds2 = pd.read_csv("submissions/xgb_1_1.csv")
preds3 = pd.read_csv("submissions/xgb_1_2.csv")
preds4 = pd.read_csv("submissions/xgb_1_3.csv")
preds5 = pd.read_csv("submissions/xgb_1_4.csv")

preds['Approved'] = rankdata(preds['Approved'], method='ordinal')
preds2['Approved'] = rankdata(preds2['Approved'], method='ordinal')
preds3['Approved'] = rankdata(preds3['Approved'], method='ordinal')
preds4['Approved'] = rankdata(preds4['Approved'], method='ordinal')
preds4['Approved'] = rankdata(preds5['Approved'], method='ordinal')

preds['Approved'] = 0.3 * (preds['Approved'] + 
                            preds2['Approved'] + 
                            preds3['Approved'] + 
                            preds4['Approved'] + 
                            preds5['Approved'])

preds.to_csv("submissions/ensembles/xgb1_ens.csv", index = False)

#### Combining predictions - xgboost2

In [95]:
preds = pd.read_csv("submissions/xgb_2.csv")
preds['ID'] = test_ids['ID']

preds2 = pd.read_csv("submissions/xgb_2_1.csv")
preds3 = pd.read_csv("submissions/xgb_2_2.csv")
preds4 = pd.read_csv("submissions/xgb_2_3.csv")
preds5 = pd.read_csv("submissions/xgb_2_4.csv")

preds['Approved'] = rankdata(preds['Approved'], method='ordinal')
preds2['Approved'] = rankdata(preds2['Approved'], method='ordinal')
preds3['Approved'] = rankdata(preds3['Approved'], method='ordinal')
preds4['Approved'] = rankdata(preds4['Approved'], method='ordinal')
preds4['Approved'] = rankdata(preds5['Approved'], method='ordinal')

preds['Approved'] = 0.3 * (preds['Approved'] + 
                            preds2['Approved'] + 
                            preds3['Approved'] + 
                            preds4['Approved'] + 
                            preds5['Approved'])

preds.to_csv("submissions/ensembles/xgb2_ens.csv", index = False)

## Ensemble

In [96]:
xgb1_pred = pd.read_csv("submissions/ensembles/xgb1_ens.csv") #XGB
xgb2_pred = pd.read_csv("submissions/ensembles/xgb2_ens.csv") #XGB
rf_pred = pd.read_csv("submissions/ensembles/rf_ens.csv") #RF


ens = xgb1_pred.copy()
ens.rename(columns={'Approved':'XGB1'}, inplace = True)
ens['XGB2'] = xgb2_pred['Approved']
ens['RF'] = rf_pred['Approved']


ens['XGB1_Rank'] = rankdata(ens['XGB1'], method='min')
ens['XGB2_Rank'] = rankdata(ens['XGB2'], method='min')

ens['XGB_Rank'] = 0.5 * ens['XGB1_Rank'] + 0.5 * ens['XGB2_Rank']
ens['RF_Rank'] = rankdata(ens['RF'], method='min')

ens['Final'] = 0.75*ens['XGB_Rank'] + 0.25*ens['RF_Rank']

ens = ens[['ID', 'Final']]
ens.rename(columns={'Final':'Approved'}, inplace = True)
ens.sort_index(inplace = True)
ens.head()

ens.to_csv("ensemble_rank.csv", index = False)

In [103]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.01, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 1200

params['seed'] = 123
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
#xgb.cv(params, dtrain, num_rounds, nfold=4)
#exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_3.csv", index = False)

In [104]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.01, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 1200

params['seed'] = 1234
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
#xgb.cv(params, dtrain, num_rounds, nfold=4)
#exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_3_1.csv", index = False)

In [105]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.01, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 1200

params['seed'] = 12345
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
#xgb.cv(params, dtrain, num_rounds, nfold=4)
#exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_3_2.csv", index = False)

In [106]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.01, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 1200

params['seed'] = 123456
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
#xgb.cv(params, dtrain, num_rounds, nfold=4)
#exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_3_3.csv", index = False)

In [107]:
params = {'booster':'gbtree', 'objective':'binary:logistic', 'max_depth':9, 'eval_metric':'auc',
          'eta':0.01, 'silent':1, 'nthread':4, 'subsample': 0.9, 'colsample_bytree':0.9, 'scale_pos_weight': 1,
          'min_child_weight':3, 'max_delta_step':3}
num_rounds = 1200

params['seed'] = 1234567
dtrain = xgb.DMatrix(X_train, y_train, missing=np.nan)
#xgb.cv(params, dtrain, num_rounds, nfold=4)
#exit()

clf = xgb.train(params, dtrain, num_rounds)
dtest = xgb.DMatrix(X_test, missing = np.nan)
test_preds_xgb = clf.predict(dtest)

submission = pd.DataFrame({ 'ID':test_ids, 'Approved':test_preds_xgb})
submission = submission[['ID', 'Approved']]
submission.to_csv("submissions/xgb_3_4.csv", index = False)

In [ ]:
preds = pd.read_csv("submissions/xgb_3.csv")
preds['ID'] = test_ids['ID']

preds2 = pd.read_csv("submissions/xgb_3_1.csv")
preds3 = pd.read_csv("submissions/xgb_3_2.csv")
preds4 = pd.read_csv("submissions/xgb_3_3.csv")
preds5 = pd.read_csv("submissions/xgb_3_4.csv")

preds['Approved'] = rankdata(preds['Approved'], method='ordinal')
preds2['Approved'] = rankdata(preds2['Approved'], method='ordinal')
preds3['Approved'] = rankdata(preds3['Approved'], method='ordinal')
preds4['Approved'] = rankdata(preds4['Approved'], method='ordinal')
preds4['Approved'] = rankdata(preds5['Approved'], method='ordinal')

preds['Approved'] = 0.3 * (preds['Approved'] + 
                            preds2['Approved'] + 
                            preds3['Approved'] + 
                            preds4['Approved'] + 
                            preds5['Approved'])

preds.to_csv("submissions/ensembles/xgb3_ens.csv", index = False)